# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# importing output.csv from 
weather_df = pd.read_csv("..\output_data\WeatherPy_Output\weather.csv")
weather_df

,Unnamed: 0,City,Lat,Lng,Max Temp (f),Humidity,Cloudiness,Wind Speed
0,0,vaini,-21.2000,-175.2000,75.470,100,75,5.14
1,1,hermanus,-34.4187,19.2345,49.262,93,50,2.19
2,2,rikitea,-23.1203,-134.9692,76.892,66,77,5.37
3,3,atuona,-9.8000,-139.0333,79.808,68,5,1.97
4,4,georgetown,5.4112,100.3354,82.670,88,20,2.06
...,...,...,...,...,...,...,...,...
561,561,adrar,20.5022,-10.0711,79.610,31,5,4.64
562,562,bouna,9.2667,-3.0000,82.724,60,100,3.38
563,563,felanitx,39.4696,3.1483,57.470,94,40,1.54
564,564,wynyard,-41.0000,145.7167,61.610,76,23,2.26


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configuring gmaps
gmaps.configure(api_key=g_key)

# Creating list of coordinates
coordinates = weather_df[['Lat', 'Lng']]
humidity = weather_df['Humidity']

# Plotting map and adding heat layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
                                 dissipating=False, max_intensity=60,
                                 point_radius=1.5)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
# Creating my ideal conditions

ideal_df = weather_df[weather_df['Max Temp (f)'] < 84]
ideal_df = ideal_df[(ideal_df['Wind Speed'] < 20) & (ideal_df['Wind Speed'] > 4)]
ideal_df = ideal_df[(ideal_df['Cloudiness'] < 40) & (ideal_df['Cloudiness'] > 5)]
ideal_df

,Unnamed: 0,City,Lat,Lng,Max Temp (f),Humidity,Cloudiness,Wind Speed
30,30,arraial do cabo,-22.9661,-42.0278,72.122,72,9,4.19
31,31,ancud,-41.8697,-73.8203,57.272,94,20,8.23
39,39,kapaa,22.0752,-159.3190,79.070,57,20,4.12
46,46,hobart,-42.8794,147.3294,65.264,77,20,4.12
48,48,vung tau,10.3460,107.0843,83.948,73,33,7.70
84,84,hasaki,35.7333,140.8333,73.274,46,20,4.12
94,94,abatskoye,56.2875,70.4555,41.558,68,7,4.38
95,95,praia,14.9215,-23.5087,71.870,73,20,5.66
103,103,bathsheba,13.2167,-59.5167,77.270,65,20,7.72
155,155,bluff,-46.6000,168.3333,54.266,74,28,6.71


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [24]:
# Creating hotel data frame and adding blank "Hotel Name" column
hotel_df = ideal_df
hotel_df['Hotel Name'] = ''

hotel_name = []
hotel_lat = []
hotel_lng = []

# Creating for loop to add hotel for each row
for i, row in hotel_df.iterrows():

    try:
        # Setting search parameters and city coordinates
        hotel_search = "Hotel"
        city_coords = f"{row[2]},{row[3]}"
        radius = 5000

        params = {
            "location": city_coords,
            "keyword": hotel_search,
            "radius": radius,
            "key": g_key
        }

        # Building URL for Google Maps API
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        response = requests.get(base_url, params)


        hotel_choice = response.json()

    

        hotel_name.append(hotel_choice['results'][0]['name'])
        hotel_lat.append(hotel_choice['results'][0]['geometry']['location']['lat'])
        hotel_lng.append(hotel_choice['results'][0]['geometry']['location']['lng'])
    except:
        print("Hotel name not found")


hotel_df['Hotel Name'] = hotel_name
hotel_df

Hotel name not found
Hotel name not found


ValueError: Length of values does not match length of index

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
